In [2]:
import requests
import pandas as pd
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki)
df_tor = pd.read_html(wiki_page.content, header=0)[0]
df_new = df_tor[df_tor.Borough != 'Not assigned']
df_new.loc[df_new.Neighbourhood == 'Not assigned']
df_new.Neighbourhood.replace('Not assigned',df_new.Borough,inplace=True)
df_toronto = df_new.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))
df_toronto = df_toronto.reset_index()
df_toronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df_toronto.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
df_toronto.shape
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_toronto = df_toronto.join(df_geo.set_index('Postal Code'), on='PostalCode')
df_toronto.head(5)

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

Downloading additional packages for map and geocoding

In [6]:
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium --yes
!conda install -c conda-forge geopy --yes
import geocoder
import folium
import geopy
from geopy.geocoders import Nominatim 

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
print('All packages are available and loaded')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/petervonogs/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50 

Creation of the map for Toronto

In [7]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

Add Fouresquare Credentials and Version

In [8]:
CLIENT_ID = 'MME5I1AR5WPEIJEYMN3S1AL4EQTCNOKSKR5JK2A2J4QRLU2X' # your Foursquare ID
CLIENT_SECRET = 'FXWWICGHB2GUMC5XSBJOMT5EHRMDRUETQFCE1MRFYE2YNEYO' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MME5I1AR5WPEIJEYMN3S1AL4EQTCNOKSKR5JK2A2J4QRLU2X
CLIENT_SECRET:FXWWICGHB2GUMC5XSBJOMT5EHRMDRUETQFCE1MRFYE2YNEYO


Prepare long/lat and name

In [10]:
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


Generate the API call - Url

In [11]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MME5I1AR5WPEIJEYMN3S1AL4EQTCNOKSKR5JK2A2J4QRLU2X&client_secret=FXWWICGHB2GUMC5XSBJOMT5EHRMDRUETQFCE1MRFYE2YNEYO&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Showing results in json format

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2cfdc888ec7f202b854bcf'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

Function to extract the category of the venue

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories        lat        lng
0  Wendy’s  Fast Food Restaurant  43.807448 -79.199056

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(2156, 7)


Neighbourhood  Neighborhood Latitude  \
0                          Malvern, Rouge              43.806686   
1  Rouge Hill, Port Union, Highland Creek              43.784535   
2       Guildwood, Morningside, West Hill              43.763573   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497  Royal Canadian Legion       43.782533   
2              -79.188711         RBC Royal Bank       43.766790   
3              -79.188711      G & G Electronics       43.765309   
4              -79.188711             Sail Sushi       43.765951   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191151                  Bank  
3       -79.191537     Electronics Store  
4       -79.191275            Restaurant

In [21]:
toronto_venues.groupby('Neighbourhood').count()

Neighborhood Latitude  \
Neighbourhood                                                            
Agincourt                                                            4   
Alderwood, Long Branch                                               8   
Bathurst Manor, Wilson Heights, Downsview North                     23   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   27   
...                                                                ...   
Willowdale, Willowdale East                                         34   
Willowdale, Willowdale West                                          4   
Woburn                                                               4   
Woodbine Heights                                                     8   
York Mills West                                                      2   

                                                 Neighborhood Longitude  \
Neighbourhood                                                             
Agincourt                                                             4   
Alderwood, Long Branch                                                8   
Bathurst Manor, Wilson Heights, Downsview North                      23   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    27   
...                                                                 ...   
Willowdale, Willowdale East                                          34   
Willowdale, Willowdale West                                           4   
Woburn                                                                4   
Woodbine Heights                                                      8   
York Mills West                                                       2   

                                                 Venue  Venue Latitude  \
Neighbourhood                                                            
Agincourt                                            4               4   
Alderwood, Long Branch                               8               8   
Bathurst Manor, Wilson Heights, Downsview North     23              23   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   27              27   
...                                                ...             ...   
Willowdale, Willowdale East                         34              34   
Willowdale, Willowdale West                          4               4   
Woburn                                               4               4   
Woodbine Heights                                     8               8   
York Mills West                                      2               2   

                                                 Venue Longitude  \
Neighbourhood                                                      
Agincourt                                                      4   
Alderwood, Long Branch                                         8   
Bathurst Manor, Wilson Heights, Downsview North               23   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             27   
...                                                          ...   
Willowdale, Willowdale East                                   34   
Willowdale, Willowdale West                                    4   
Woburn                                                         4   
Woodbine Heights                                               8   
York Mills West                                                2   

                                                 Venue Category  
Neighbourhood                                                    
Agincourt                                                     4  
Alderwood, Long Branch                                        8 

How many categories are available?

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighbourhood  Accessories Store  \
0                          Malvern, Rouge                  0   
1  Rouge Hill, Port Union, Highland Creek                  0   
2       Guildwood, Morningside, West Hill                  0   
3       Guildwood, Morningside, West Hill                  0   
4       Guildwood, Morningside, West Hill                  0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                  0        0                   0               0   
1                  0        0                   0               0   
2                  0        0                   0               0   
3                  0        0                   0               0   
4                  0        0                   0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  ...  \
0                0                 0                    0             0  ...   
1                0                 0                    0             0  ...   
2                0                 0                    0             0  ...   
3                0                 0                    0             0  ...   
4                0                 0                    0             0  ...   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  \
0                      0                0         0          0            0   
1                      0                0         0          0            0   
2                      0                0         0          0            0   
3                      0                0         0          0            0   
4                      0                0         0          0            0   

   Women's Store  Yoga Studio  
0              0            0  
1              0            0  
2              0            0  
3              0            0  
4              0            0  

[5 rows x 273 columns]

In [23]:
toronto_onehot.shape

(2156, 273)

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

Neighbourhood  Accessories Store  \
0                                        Agincourt                0.0   
1                           Alderwood, Long Branch                0.0   
2  Bathurst Manor, Wilson Heights, Downsview North                0.0   
3                                  Bayview Village                0.0   
4                Bedford Park, Lawrence Manor East                0.0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                0.0      0.0                 0.0             0.0   
1                0.0      0.0                 0.0             0.0   
2                0.0      0.0                 0.0             0.0   
3                0.0      0.0                 0.0             0.0   
4                0.0      0.0                 0.0             0.0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  ...  \
0              0.0               0.0             0.000000           0.0  ...   
1              0.0               0.0             0.000000           0.0  ...   
2              0.0               0.0             0.000000           0.0  ...   
3              0.0               0.0             0.000000           0.0  ...   
4              0.0               0.0             0.037037           0.0  ...   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                            0.0               0.0          0.0   
1                            0.0               0.0          0.0   
2                            0.0               0.0          0.0   
3                            0.0               0.0          0.0   
4                            0.0               0.0          0.0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  \
0                    0.0              0.0       0.0        0.0          0.0   
1                    0.0              0.0       0.0        0.0          0.0   
2                    0.0              0.0       0.0        0.0          0.0   
3                    0.0              0.0       0.0        0.0          0.0   
4                    0.0              0.0       0.0        0.0          0.0   

   Women's Store  Yoga Studio  
0            0.0          0.0  
1            0.0          0.0  
2            0.0          0.0  
3            0.0          0.0  
4            0.0          0.0  

[5 rows x 273 columns]

In [25]:
toronto_grouped.shape

(95, 273)

Function for most common venues in the neighborhood

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

calling function with top 10 venues to be shown

In [29]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East    Italian Restaurant   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot          Skating Rink   
1                Coffee Shop        Sandwich Place              Pharmacy   
2                       Bank         Shopping Mall        Sandwich Place   
3         Chinese Restaurant                  Bank   Japanese Restaurant   
4                Coffee Shop            Restaurant        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
0   Dumpling Restaurant   Distribution Center                      Dog Run   
1                  Pool                   Pub                          Gym   
2            Restaurant           Supermarket               Ice Cream Shop   
3  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   
4             Juice Bar            Hobby Shop                     Pharmacy   

  8th Most Common Venue      9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                 Donut Shop              Drugstore  
1      Airport Terminal               Dessert Shop     Falafel Restaurant  
2      Sushi Restaurant  Middle Eastern Restaurant      Mobile Phone Shop  
3   Dumpling Restaurant                  Drugstore     Dim Sum Restaurant  
4           Pizza Place                        Pub                   Café

ClusterNeighborhoods using k-mean

In [38]:

from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() 

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353             2.0  Fast Food Restaurant             Drugstore   
1 -79.160497             4.0                   Bar           Yoga Studio   
2 -79.188711             0.0          Intersection     Electronics Store   
3 -79.216917             0.0           Coffee Shop          Soccer Field   
4 -79.239476             0.0                  Bank      Hakka Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                 Diner        Discount Store   Distribution Center   
1             Drugstore        Discount Store   Distribution Center   
2        Breakfast Spot            Restaurant   Rental Car Location   
3     Korean Restaurant           Yoga Studio             Drugstore   
4   Fried Chicken Joint  Caribbean Restaurant       Thai Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0               Dog Run      Doner Restaurant            Donut Shop   
1               Dog Run      Doner Restaurant            Donut Shop   
2        Medical Center                  Bank    Mexican Restaurant   
3        Discount Store   Distribution Center               Dog Run   
4    Athletics & Sports           Gas Station                Bakery   

  9th Most Common Venue 10th Most Common Venue  
0   Dumpling Restaurant          Women's Store  
1   Dumpling Restaurant                  Field  
2           Yoga Studio         Discount Store  
3      Doner Restaurant             Donut Shop  
4               Dog Run                  Diner

In [40]:
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

PostalCode      Borough                                       Neighborhood  \
16        M1X  Scarborough                                        Upper Rouge   
20        M2L   North York                           York Mills, Silver Hills   
93        M9A    Etobicoke            Islington Avenue, Humber Valley Village   
94        M9B    Etobicoke  West Deane Park, Princess Gardens, Martin Grov...   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
16  43.836125 -79.205636             NaN                   NaN   
20  43.757490 -79.374714             NaN                   NaN   
93  43.667856 -79.532242             NaN                   NaN   
94  43.650943 -79.554724             NaN                   NaN   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
16                   NaN                   NaN                   NaN   
20                   NaN                   NaN                   NaN   
93                   NaN                   NaN                   NaN   
94                   NaN                   NaN                   NaN   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
16                   NaN                   NaN                   NaN   
20                   NaN                   NaN                   NaN   
93                   NaN                   NaN                   NaN   
94                   NaN                   NaN                   NaN   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
16                   NaN                   NaN                    NaN  
20                   NaN                   NaN                    NaN  
93                   NaN                   NaN                    NaN  
94                   NaN                   NaN                    NaN

In [41]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Detailed analysis Clusters 1

In [42]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
2    Scarborough             0.0          Intersection     Electronics Store   
3    Scarborough             0.0           Coffee Shop          Soccer Field   
4    Scarborough             0.0                  Bank      Hakka Restaurant   
5    Scarborough             0.0         Women's Store            Playground   
6    Scarborough             0.0    Chinese Restaurant      Department Store   
..           ...             ...                   ...                   ...   
96    North York             0.0                   Gym           Pizza Place   
99     Etobicoke             0.0           Pizza Place    Chinese Restaurant   
100    Etobicoke             0.0                  Park     Mobile Phone Shop   
101    Etobicoke             0.0         Grocery Store           Pizza Place   
102    Etobicoke             0.0         Garden Center   Rental Car Location   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2          Breakfast Spot            Restaurant   Rental Car Location   
3       Korean Restaurant           Yoga Studio             Drugstore   
4     Fried Chicken Joint  Caribbean Restaurant       Thai Restaurant   
5               Drugstore                 Diner        Discount Store   
6          Discount Store           Coffee Shop     Convenience Store   
..                    ...                   ...                   ...   
96          Grocery Store      Department Store           Event Space   
99         Sandwich Place           Coffee Shop          Intersection   
100        Sandwich Place           Yoga Studio      Doner Restaurant   
101   Fried Chicken Joint              Pharmacy        Discount Store   
102             Drugstore           Yoga Studio               Dog Run   

    6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
2          Medical Center                  Bank           Mexican Restaurant   
3          Discount Store   Distribution Center                      Dog Run   
4      Athletics & Sports           Gas Station                       Bakery   
5     Distribution Center               Dog Run             Doner Restaurant   
6              Hobby Shop           Yoga Studio          Distribution Center   
..                    ...                   ...                          ...   
96   Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   
99         Discount Store           Yoga Studio                        Diner   
100                 Diner        Discount Store          Distribution Center   
101        Sandwich Place            Beer Store         Fast Food Restaurant   
102    Dim Sum Restaurant                 Diner               Discount Store   

    9th Most Common Venue 10th Most Common Venue  
2             Yoga Studio         Discount Store  
3        Doner Restaurant             Donut Shop  
4                 Dog Run                  Diner  
5              Donut Shop    Dumpling Restaurant  
6                 Dog Run       Doner Restaurant  
..                    ...                    ...  
96    Dumpling Restaurant              Drugstore  
99    Distribution Center                Dog Run  
100               Dog Run             Donut Shop  
101           Video Store      German Restaurant  
102   Distribution Center       Doner Restaurant  

[89 rows x 12 columns]

Detailed analysis Clusters 2

In [51]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
14       Scarborough             1.0                  Park   
21        North York             1.0                  Park   
23        North York             1.0                  Park   
50  Downtown Toronto             1.0                  Park   
74              York             1.0                  Park   
98              York             1.0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14            Playground           Yoga Studio            Donut Shop   
21           Yoga Studio             Drugstore                 Diner   
23     Convenience Store           Yoga Studio             Drugstore   
50            Playground                 Trail      Doner Restaurant   
74         Women's Store                  Pool            Donut Shop   
98           Yoga Studio             Drugstore                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14    Dim Sum Restaurant                 Diner        Discount Store   
21        Discount Store   Distribution Center               Dog Run   
23                 Diner        Discount Store   Distribution Center   
50    Dim Sum Restaurant                 Diner        Discount Store   
74    Dim Sum Restaurant                 Diner        Discount Store   
98        Discount Store   Distribution Center               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
14   Distribution Center               Dog Run       Doner Restaurant  
21      Doner Restaurant            Donut Shop    Dumpling Restaurant  
23               Dog Run      Doner Restaurant             Donut Shop  
50   Distribution Center               Dog Run             Donut Shop  
74   Distribution Center               Dog Run       Doner Restaurant  
98      Doner Restaurant            Donut Shop    Dumpling Restaurant

Detailed analysis Clusters 3

In [52]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0  Scarborough             2.0  Fast Food Restaurant             Drugstore   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                 Diner        Discount Store   Distribution Center   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0               Dog Run      Doner Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0   Dumpling Restaurant          Women's Store

Detailed analysis Clusters 4

In [53]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
91   Etobicoke             3.0        Baseball Field           Yoga Studio   
97  North York             3.0          Food Service        Baseball Field   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
91             Drugstore        Discount Store   Distribution Center   
97           Yoga Studio   Distribution Center               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
91               Dog Run      Doner Restaurant            Donut Shop   
97      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue 10th Most Common Venue  
91   Dumpling Restaurant                  Field  
97   Dumpling Restaurant                  Diner

Detailed analysis Clusters 5

In [54]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
1  Scarborough             4.0                   Bar           Yoga Studio   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
1             Drugstore        Discount Store   Distribution Center   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
1               Dog Run      Doner Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
1   Dumpling Restaurant                  Field